In [ ]:
import math
import random

def mod_inverse(a, m):
    try:
        return pow(a, -1, m)
    except ValueError:
        return None

def is_coprime(a, b):
    return math.gcd(a, b) == 1

# ---------------- Affine Cipher ----------------
def affine_encrypt(text, a, b):
    if not is_coprime(a, 26):
        raise ValueError("Multiplicative key 'a' must be coprime to 26.")
    
    encrypted_text = ""
    for char in text:
        if char.isalpha():
            base = ord('A') if char.isupper() else ord('a')
            x = ord(char) - base
            encrypted_char = (a * x + b) % 26
            encrypted_text += chr(encrypted_char + base)
        else:
            encrypted_text += char
    return encrypted_text

def affine_decrypt(ciphertext, a, b):
    if not is_coprime(a, 26):
        raise ValueError("Multiplicative key 'a' must be coprime to 26.")
    
    a_inv = mod_inverse(a, 26)
    if a_inv is None:
        raise ValueError("No modular inverse found for 'a'. Choose a different key.")
    
    decrypted_text = ""
    for char in ciphertext:
        if char.isalpha():
            base = ord('A') if char.isupper() else ord('a')
            y = ord(char) - base
            decrypted_char = (a_inv * (y - b)) % 26
            decrypted_text += chr(decrypted_char + base)
        else:
            decrypted_text += char
    return decrypted_text

# ---------------- RSA Encryption ----------------
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

def generate_keys():
    primes = [i for i in range(50, 200) if is_prime(i)]
    p, q = random.sample(primes, 2)
    
    n = p * q
    phi = (p - 1) * (q - 1)

    e = next(i for i in range(2, phi) if is_coprime(i, phi) and mod_inverse(i, phi) is not None)
    d = mod_inverse(e, phi)
    
    return (e, n), (d, n)

def rsa_encrypt(text, public_key):
    e, n = public_key
    return [pow(ord(char), e, n) for char in text]

def rsa_decrypt(ciphertext, private_key):
    d, n = private_key
    return ''.join([chr(pow(char, d, n)) for char in ciphertext])

# ---------------- Hybrid Encryption Model ----------------
def hybrid_encrypt(text, a, b, public_key):
    affine_encrypted = affine_encrypt(text, a, b)
    rsa_encrypted = rsa_encrypt(affine_encrypted, public_key)
    return rsa_encrypted

def hybrid_decrypt(ciphertext, a, b, private_key):
    rsa_decrypted = rsa_decrypt(ciphertext, private_key)
    affine_decrypted = affine_decrypt(rsa_decrypted, a, b)
    return affine_decrypted

# ---------------- Interactive Execution ----------------
if __name__ == "__main__":
    private_key = None  # Store private key for reuse
    while True:
        print("\n--- Hybrid Encryption System ---")
        print("1. Encrypt")
        print("2. Decrypt")
        print("3. Exit")
        choice = input("Enter your choice: ")
        
        if choice == "1":
            text = input("Enter text to encrypt: ")
            a = int(input("Enter multiplicative key (coprime to 26): "))
            b = int(input("Enter additive key: "))
            
            if not is_coprime(a, 26):
                print("Error: Invalid 'a', must be coprime to 26.")
                continue
            
            public_key, private_key = generate_keys()
            encrypted_text = hybrid_encrypt(text, a, b, public_key)
            print("Encrypted Output:", ','.join(map(str, encrypted_text)))
            print("Private Key for Decryption:", private_key)
        
        elif choice == "2":
            if private_key is None:
                print("Error: No stored private key. Please provide manually.")
                continue
            
            text = input("Enter encrypted numbers (comma-separated): ")
            try:
                encrypted_numbers = list(map(int, text.split(",")))
                a = int(input("Enter multiplicative key (coprime to 26): "))
                b = int(input("Enter additive key: "))
                private_key = tuple(map(int, input("Enter private key (d, n): ").split(",")))
                
                decrypted_text = hybrid_decrypt(encrypted_numbers, a, b, private_key)
                print("Decrypted Output:", decrypted_text)
            except ValueError:
                print("Invalid input format. Ensure encrypted numbers are comma-separated and valid.")
        
        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Invalid choice, please try again.")



--- Hybrid Encryption System ---
1. Encrypt
2. Decrypt
3. Exit
